In [2]:
#https://docs.sqlalchemy.org/en/latest/core/engines.html
#SET UP LOCAL CONNECTION TO DATABASE FILE
from sqlalchemy import create_engine
engine = create_engine('sqlite:///C:\\Users\\e333775\\Documents\\BA510\\final-project-team-mike-jose-trish\\docs\\csv_data\\CoursesDB.db')
import pandas as pd


In [5]:
#list of tables in OPERATIONAL_DB
pd.read_sql('select * from instructors',engine).head(1)
pd.read_sql('select * from locations',engine).head(1) 
pd.read_sql('select * from course_titles',engine).head(1)
pd.read_sql('select * from courses',engine).head(1)
pd.read_sql('select * from catalog',engine).head(1)
pd.read_sql("""select * from meetings""",engine).head(1)

term    crn location day                start                  end
0  Fall2014  70384  DSB 105   T  2014-09-02 08:00:00  2014-09-02 09:15:00

In [9]:
#READ IN INSTRUCTOR DATA FROM EXCEL
instructors = pd.read_excel('courses_combined.xlsx',sheet_name = 'instructors')

#CREATE TABLE INSTRUCTORS
pd.io.sql.execute("""CREATE TABLE instructors (instructor_id INTEGER, primary_instructor varchar(20))""",engine)

#FOR EACH ROW IN CSV, INSERT INTO INSTRUCTORS TABLE
for row in instructors.itertuples():
    pd.io.sql.execute("""INSERT INTO instructors (instructor_id, primary_instructor) VALUES ({},'{}')""".format(row[1],row[2]),engine)


In [13]:
#READ IN THE LOCATIONS
locations = pd.read_excel('course_meetings_combined.xlsx',sheetname='locations_table')

#CREATE LOCATIONS TABLE
pd.io.sql.execute("""CREATE TABLE IF NOT EXISTS locations (NAME TEXT, BUILDING TEXT, ROOM TEXT)""",engine)

#FOR EACH ROW IN EXCEL, INSERT INTO LOCATIONS TABLES
for row in locations.itertuples():
    pd.io.sql.execute("""INSERT INTO locations (NAME,BUILDING,ROOM) VALUES ('{}','{}','{}')""".format(row[1],row[2],row[3]),engine)

In [10]:
#READ IN COURSE_TITLE DATA FROM EXCEL
course_titles = pd.read_excel('courses_combined.xlsx',sheet_name = 'course_titles')

#CREATE TABLE COURSE_TITLES
pd.io.sql.execute("""CREATE TABLE course_titles (course_id INTEGER, course_title varchar(100))""",engine)

#FOR EACH ROW IN EXCEL FILE, INSERT INTO COURSE_TITLES TABLE
for row in course_titles.itertuples():
    pd.io.sql.execute("""INSERT INTO course_titles (course_id, course_title) VALUES ({},'{}')""".format(row[1],row[2]),engine)


In [11]:
#READ IN COURSES DATA FROM EXCEL
courses_combined = pd.read_excel('courses_combined.xlsx',sheet_name='courses_combined').drop(['meetings','timecodes'],1)

#CREATE COURSES TABLE
pd.io.sql.execute("""CREATE TABLE  courses (term TEXT, crn INTEGER, catalog_id TEXT, section TEXT, credits Integer, title_id Integer,instructor_id Integer, cap Integer, act Integer, rem Integer)""",engine)

#FOR EACH ROW IN EXCEL FILE, INSERT INTO COURSES TABLE
for row in courses_combined.itertuples():
    pd.io.sql.execute("""INSERT INTO courses (term,crn,catalog_id,section,credits,title_id,instructor_id,cap,act,rem) VALUES('{}',{},'{}','{}',{},{},{},{},{},{})""".format(row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10]),engine)


In [12]:
#READ IN THE CATALOG FILES
catalog_df = pd.read_csv('catalog_files_combined.csv').drop('Unnamed: 0',1)

#CREATE CATALOG TABLE
pd.io.sql.execute("""CREATE TABLE catalog (program_code TEXT, program_name TEXT, catalog_id TEXT, course_title TEXT, credits TEXT, prereqs TEXT, coreqs TEXT, fees TEXT, attributes TEXT, description TEXT)""",engine)

#FOR EACH ROW IN CSV FILE, INSERT INTO CATALOG TABLE
for row in catalog_df.itertuples():
    pd.io.sql.execute("""INSERT INTO catalog (program_code,program_name,catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description) VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}')""".format(row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10]),engine)
#catalog_df    

In [9]:
#CREATE THE MEETINGS TABLE 
pd.io.sql.execute("""CREATE TABLE IF NOT EXISTS meetings (
 term TEXT,
 crn INTEGER,
 location TEXT,
 day TEXT,
 start TEXT,
 end TEXT
)""",engine)

#READ DATA IN FROM CSV THAT WE WILL INSERT
df = pd.read_csv('course_meetings_combined.csv').drop('Unnamed: 0',1)

#USE PYTHON TO MODIFY THE FORMAT OF THE START AND END COLUMNS
df['start'] = pd.to_datetime(df['start'])
df['end'] = pd.to_datetime(df['end'])

#FOR EACH ROW IN THE CSV FILE, INSERT THE VALUES INTO THE MEETINGS TABLE
import time 
start = time.time()
count = 0
for row in df.itertuples():
    if count % 50000 == 0:
        print(count)
    pd.io.sql.execute("""INSERT INTO meetings (term,crn,location,day,start,end) VALUES ('{}',{},'{}','{}','{}','{}')""".format(row[1],row[2],row[3],row[4],row[5],row[6]),engine)
    count = count+1
end = time.time()

0
50000
100000
150000
200000
250000
300000


In [12]:
##IT TOOK 50 MINUTES TO INSERT ~315K ROWS
print((end-start)/60)

50.050183312098184


In [14]:
pd.read_sql("""SELECT
                b.primary_instructor,
                c.course_title, 
                a.term,a.crn,
                a.catalog_id,
                a.section,
                a.credits,
                a.cap,
                a.act,
                a.rem 
                FROM courses a, instructors b, course_titles c 
                WHERE a.instructor_id = b.instructor_id 
                AND a.title_id = c.course_id""",engine).head(1)

primary_instructor                          course_title      term    crn  \
0  Michael P.  Coyne  Introduction to Financial Accounting  Fall2014  70384   

  catalog_id section  credits  cap  act  rem  
0    AC 0011     C01      3.0    0   31  -31

In [16]:
###DOMAIN INTEGRITY TEST. ARE CREDITS COMING THROUGH CORRECTLY AS INTEGERS?
###MOST ARE INTEGERS, BUT SOME RECORDS ARE FRACTIONAL (1/3 or 1/2 or 3/6) MEANT TO INDICATE 1 OR 3
###THESE ARE COMING THROUGH AS FRACTIONS
###THESE ARE BAD RECORDS THAT WILL BE IGNORED IN ANALYSIS

pd.read_sql('select * from courses where typeof(credits) <> "integer"',engine).head()

term    crn catalog_id section   credits  title_id  instructor_id  \
0    Fall2014  72962    PS 0399       A  0.500000        19            454   
1    Fall2015  77120    PS 0399      AB  0.500000        19            454   
2    Fall2018  74684   CN 0590S       1  0.500000       163            126   
3  Summer2016  56298   ENW 0346       1  0.333333        10            218   
4  Spring2016  39190    PO 0298       A  0.750000        10            467   

   cap  act  rem  
0    0    1   -1  
1    0    1   -1  
2    0    6   -6  
3    0    4   -4  
4    0    4   -4

In [21]:
###DOMAIN INTEGRITY. ARE THESE THE DAYS WE EXPECT MONDAY-SUNDAY?
###SEEMS TO BE OKAY, SOME CLASSES ON SUNDAY (U)
pd.read_sql("""select distinct DAY from meetings group by day""",engine)

day
0   F
1   M
2   R
3   S
4   T
5   U
6   W

In [22]:
###DOMAIN INTEGRITY IS THIS PROVES THERE IS ONLY 1 INSTANCE OF EACH UNIQUE CRN PER TERM
pd.read_sql("""SELECT max(count) 
                 FROM (SELECT crn,term,count(*) as count 
                         FROM courses 
                         GROUP BY crn,term 
                         ORDER BY 3 desc)""",engine)

max(count)
0           1

In [27]:
###REFERENTIAL INTEGRITY. ARE THERE CATALOG ID NUMBERS THAT DO NOT SHOW UP IN CATALOG TABLE?
###THERE APPEARS TO BE 200+ CATALOG ID NUMBERS IN THE COURSE OFFERINGS THAT DO NOT SHOW UP IN THE CATALOG TABLE
###tHIS IS LIKELY BECAUSE WEIRD LETTERS HAVE BEEN APPENDED OR HUMAN ERROR / MISSING DATA 
###THESE INSTANCES WILL BE IGNORED IN ANALYSIS

pd.read_sql("""
            SELECT distinct a.catalog_id, b.program_name 
            FROM courses a 
            LEFT JOIN catalog b
            ON a.catalog_id = b.catalog_id 
            WHERE b.program_name is null
            """,engine).head()

catalog_id program_name
0    AH 0012         None
1    CO 0398         None
2    CS 0377         None
3    MD 0099         None
4   MD 0403A         None

In [25]:
pd.read_sql("""select * from catalog where catalog_id like 'AH 0012'""",engine).head()

Empty DataFrame
Columns: [program_code, program_name, catalog_id, course_title, credits, prereqs, coreqs, fees, attributes, description]
Index: []